In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
import os
from skimage import io, util, morphology, measure, filters, feature
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定資料夾路徑
folder_path = '/content/drive/MyDrive/CMPT898_Project/898_Files/Annotated_Dataset/LR-26_Test_Annotated/YOLODataset_Test/images/test'

# 定義 gamma 調整函數
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype('uint8')
    return cv2.LUT(image, table)

# 遍歷資料夾中的所有檔案
for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # 確保是圖片檔案
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_name}")

        # 讀取圖片
        image = io.imread(file_path)
        image_gray = util.img_as_float(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))

        # 添加遮罩區域 (黑色遮罩)
        image_gray[:180, :] = 0
        image_gray[-180:, :] = 0
        image_gray[:500, :500] = 0
        image_gray[:500, -500:] = 0

        # 直方圖均衡化 + gamma 調整
        image_gray = cv2.equalizeHist((image_gray * 255).astype('uint8'))
        image_gray = adjust_gamma(image_gray, gamma=0.03)

        # 邊緣檢測
        edges = feature.canny(image_gray, sigma=2.0)

        # 自適應閾值分割
        thresh = filters.threshold_otsu(image_gray)
        binary_mask = image_gray > thresh

        # 形態學處理
        struct_element = morphology.disk(1.2)
        mask = morphology.binary_opening(binary_mask, struct_element)
        mask = morphology.binary_closing(mask, struct_element)
        mask = morphology.remove_small_objects(mask, min_size=10, connectivity=2)

        # 標記連通區域
        labeled = measure.label(mask)
        regions = measure.regionprops(labeled)
        print(f"Number of regions found: {len(regions)}")

        if regions:
            # 找出最大的連通區域
            largest_region = max(regions, key=lambda r: r.area)
            mask = labeled == largest_region.label

            # 顯示結果
            plt.figure(figsize=(15, 10))
            plt.subplot(1, 3, 1)
            plt.imshow(image_gray, cmap='gray')
            plt.title('Grayscale Image with Mask Applied')

            plt.subplot(1, 3, 2)
            plt.imshow(binary_mask, cmap='gray')
            plt.title('Threshold Otsu')

            plt.subplot(1, 3, 3)
            plt.imshow(mask, cmap='gray')
            plt.title(f'Final {file_name}')

            plt.show()
        else:
            print(f"No regions found in {file_name}.")


Output hidden; open in https://colab.research.google.com to view.

In [52]:
import os
from skimage import io, util, morphology, measure, filters, feature
import cv2
import numpy as np
import matplotlib.pyplot as plt
folder_path = '/content/drive/MyDrive/CMPT898_Project/898_Files/Annotated_Dataset/LR-26_Test_Annotated/YOLODataset_Test/images/test'
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype('uint8')
    return cv2.LUT(image, table)

def enhance_details(image_gray):

    # 使用UnSharp Masking增強細節
    gaussian_3 = cv2.GaussianBlur(image_gray, (0, 0), 2.0)
    unsharp_image = cv2.addWeighted(image_gray, 1.5, gaussian_3, -0.5, 0)
    return unsharp_image

def adaptive_threshold(image_gray):

    # 結合全局和局部閾值
    global_thresh = filters.threshold_otsu(image_gray)
    local_thresh = filters.threshold_local(image_gray, block_size=35, method='gaussian')

    # 組合兩種閾值結果
    global_mask = image_gray > global_thresh
    local_mask = image_gray > local_thresh

    # 使用邏輯運算合併結果
    combined_mask = global_mask | local_mask
    return combined_mask

def process_image(file_path):
    # 讀取圖片
    image = io.imread(file_path)
    image_gray = util.img_as_float(cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))

    # 添加遮罩區域
    image_gray[:180, :] = 0
    image_gray[-180:, :] = 0
    image_gray[:500, :500] = 0
    image_gray[:500, -500:] = 0

    # 增強對比度和細節
    image_gray = cv2.equalizeHist((image_gray * 255).astype('uint8'))
    image_gray = adjust_gamma(image_gray, gamma=0.03)
    image_gray = enhance_details(image_gray)

    # 邊緣檢測
    edges = feature.canny(image_gray, sigma=1.5, low_threshold=0.05, high_threshold=0.15)

    # 自適應閾值分割
    binary_mask = adaptive_threshold(image_gray)

    # 溫和的形態學處理
    # 使用較小的結構元素
    small_struct = morphology.disk(0.7)
    large_struct = morphology.disk(1.2)

    # 分兩步驟進行形態學處理
    # 1. 使用小結構元素清理雜訊
    mask_fine = morphology.binary_opening(binary_mask, small_struct)
    mask_fine = morphology.binary_closing(mask_fine, small_struct)

    # 2. 使用較大結構元素連接主要結構
    mask_coarse = morphology.binary_opening(binary_mask, large_struct)
    mask_coarse = morphology.binary_closing(mask_coarse, large_struct)

    # 組合細節和主要結構
    mask_combined = mask_fine | mask_coarse

    # 移除小物體但保留細長結構
    mask_final = morphology.remove_small_objects(mask_combined, min_size=50, connectivity=2)

    # 使用骨架化來保留細節
    skeleton = morphology.skeletonize(mask_final)
    # 稍微膨脹骨架以恢復一些寬度
    dilated_skeleton = morphology.binary_dilation(skeleton, morphology.disk(0.8))

    # 將骨架和原始遮罩組合
    final_mask = mask_final | dilated_skeleton

    # 標記連通區域
    labeled = measure.label(final_mask)
    regions = measure.regionprops(labeled)
    print(f"Number of regions found: {len(regions)}")

    if regions:
        # 找出最大的連通區域
        largest_region = max(regions, key=lambda r: r.area)
        final_result = labeled == largest_region.label

        # 顯示結果
        plt.figure(figsize=(10, 10))
        plt.subplot(1, 2, 1)
        plt.imshow(image_gray, cmap='gray')
        plt.title('Enhanced Grayscale')


        plt.subplot(1, 2, 2)
        plt.imshow(final_result, cmap='gray')
        plt.title(f'Final Result: {file_name}')

        plt.tight_layout()
        plt.show()
    else:
        print(f"No regions found in {file_name}.")

# 使用示例
folder_path = '/content/drive/MyDrive/CMPT898_Project/898_Files/Annotated_Dataset/LR-26_Test_Annotated/YOLODataset_Test/images/test'

for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
        file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {file_name}")
        process_image(file_path)

Output hidden; open in https://colab.research.google.com to view.